<a href="https://colab.research.google.com/github/nissysathwika/Batch-241/blob/main/Yahoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy transformers sentence-transformers evaluate nltk rouge-score bert-score torch accelerate tf-keras


  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached xxhash-3.6.0-cp312-cp312-win_amd64.whl.metadata (13 kB)
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)
Using cached xxhash-3.6.0-cp312-cp312-win_amd64.whl (31 kB)
   ---------------------------------------- 0.0/27.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/27.7 MB ? eta -:--:--
    --------------------------------------- 0.5/27.7 MB 644.1 kB/s eta 0:00:43
   --- ------------------------------------ 2.4/27.7 MB 3.5 MB/s eta 0:00:08
   ----- ---------------------------------- 3.7/27.7 MB 5.2 MB/s eta 0:00:05
   ---------- ----------------------------- 7.1/27.7 MB 6.7 MB/s eta 0:00:04
   ----------- ---------------------------- 8.1/27.7 MB 6.5 MB/s eta 0:00:04
   ----------- ---------------------------- 8.1/27.7 MB 6.5 MB/s eta 0:00:04
   -------------- ------------------------- 10.0/27.7 MB 6.0 MB/s eta 0:00:03
   ---------------- ----------------------- 11.5/27.7 MB 6.2 MB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.1.0 which is incompatible.
streamlit 1.37.1 requires protobuf<6,>=3.20, but you have protobuf 6.33.4 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk

from transformers import AutoTokenizer, AutoModel, pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

import evaluate


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nissy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nissy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
train_path = "yahooo/10_categories_of_yahoo_answers_for_nlp_tasks_csv/train.csv"

df = pd.read_csv(train_path)

df.head()


,class_index,question_title,question_content,best_answer
0,5,why doesn't an optical mouse work on a glass t...,or even on some surfaces?,Optical mice use an LED and a camera to rapidl...
1,6,What is the best off-road motorcycle trail ?,long-distance trail throughout CA,i hear that the mojave road is amazing!<br />\...
2,3,What is Trans Fat? How to reduce that?,I heard that tras fat is bad for the body. Wh...,Trans fats occur in manufactured foods during ...
3,7,How many planes Fedex has?,I heard that it is the largest airline in the ...,according to the www.fedex.com web site:\nAir ...
4,7,"In the san francisco bay area, does it make se...",the prices of rent and the price of buying doe...,renting vs buying depends on your goals. <br /...


In [ ]:
print(df.columns)


Index(['class_index', 'question_title', 'question_content', 'best_answer'], dtype='object')


In [ ]:
df['combined_text'] = (
    df['question_title'].astype(str) + " " +
    df['question_content'].astype(str) + " " +
    df['best_answer'].astype(str)
)


In [ ]:
df[['question_title','question_content','best_answer']].head()


,question_title,question_content,best_answer
0,why doesn't an optical mouse work on a glass t...,or even on some surfaces?,Optical mice use an LED and a camera to rapidl...
1,What is the best off-road motorcycle trail ?,long-distance trail throughout CA,i hear that the mojave road is amazing!<br />\...
2,What is Trans Fat? How to reduce that?,I heard that tras fat is bad for the body. Wh...,Trans fats occur in manufactured foods during ...
3,How many planes Fedex has?,I heard that it is the largest airline in the ...,according to the www.fedex.com web site:\nAir ...
4,"In the san francisco bay area, does it make se...",the prices of rent and the price of buying doe...,renting vs buying depends on your goals. <br /...


In [ ]:
df = df.sample(10000, random_state=42)


In [ ]:
stopwords = set(nltk.corpus.stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    tokens = nltk.word_tokenize(text)
    tokens = [w for w in tokens if w not in stopwords]

    return " ".join(tokens)


In [ ]:
df['clean_text'] = df['combined_text'].apply(clean_text)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\nissy\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nissy\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
bert_model = AutoModel.from_pretrained("bert-base-uncased")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
sample_tokens = tokenizer(
    df['clean_text'].iloc[0],
    padding=True,
    truncation=True,
    return_tensors="pt"
)

sample_tokens


{'input_ids': tensor([[  101,  4669,  2711, 22437,  2481,  2102,  2131,  3276,  2183, 16660,
         10047, 12170,  4424, 10296,  2763,  3114,  2064,  2102,  2131,  3276,
          2183,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")

corpus = df['clean_text'].tolist()

corpus_embeddings = embedder.encode(
    corpus,
    show_progress_bar=True,
    batch_size=32
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\nissy\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nissy\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [ ]:
generator = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=256
)


config.json: 0.00B [00:00, ?B/s]

C:\Users\nissy\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nissy\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For 

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
user_question = input("Enter your question: ")

user_question_clean = clean_text(user_question)


Enter your question:  Who is Prime Minister of India?


In [ ]:
query_embedding = embedder.encode([user_question_clean])

similarity = cosine_similarity(query_embedding, corpus_embeddings)

best_index = np.argmax(similarity)

context = corpus[best_index]

reference_answer = df.iloc[best_index]['best_answer']


In [ ]:
zero_prompt = f"""
Answer the question using the context.

Context:
{context}

Question:
{user_question}

Answer:
"""


In [ ]:
few_prompt = f"""
Example:
Question: What is gravity?
Answer: Gravity is the force of attraction between objects.

Now answer:

Context:
{context}

Question:
{user_question}

Answer:
"""


In [ ]:
cot_prompt = f"""
Answer step by step.

Context:
{context}

Question:
{user_question}

Let's think step by step:
"""


In [ ]:
zero_ans = generator(zero_prompt)[0]['generated_text']

few_ans = generator(few_prompt)[0]['generated_text']

cot_ans = generator(cot_prompt)[0]['generated_text']


In [ ]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")


In [ ]:
bleu_score = bleu.compute(
    predictions=[zero_ans],
    references=[[reference_answer]]
)


In [ ]:
rouge_score = rouge.compute(
    predictions=[zero_ans],
    references=[reference_answer]
)


In [ ]:
bert_score = bertscore.compute(
    predictions=[zero_ans],
    references=[reference_answer],
    lang="en"
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:  10%|9         | 136M/1.42G [00:00<?, ?B/s]

C:\Users\nissy\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nissy\.cache\huggingface\hub\models--roberta-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.de

In [ ]:
print("Generated Answer:\n", zero_ans)

print("\nBLEU Score:", bleu_score)
print("\nROUGE Score:", rouge_score)

print("\nBERTScore Precision:", np.mean(bert_score['precision']))
print("BERTScore Recall:", np.mean(bert_score['recall']))
print("BERTScore F1:", np.mean(bert_score['f1']))


Generated Answer:
 nan apj abdul kalamnnhow

BLEU Score: {'bleu': 0.0, 'precisions': [0.0, 0.0, 0.0, 0.0], 'brevity_penalty': 0.014264233908999256, 'length_ratio': 0.19047619047619047, 'translation_length': 4, 'reference_length': 21}

ROUGE Score: {'rouge1': 0.125, 'rouge2': 0.0, 'rougeL': 0.125, 'rougeLsum': 0.125}

BERTScore Precision: 0.790479838848114
BERTScore Recall: 0.8034825325012207
BERTScore F1: 0.7969281077384949
